In [ ]:
pip install python-dotenv

In [ ]:
# import os
# from dotenv import load_dotenv, find_dotenv
# _= load_dotenv(find_dotenv())

# ap =os.getenv("MY_VAR")

In [ ]:
# !pip install langchain
# !pip install openai

In [ ]:
!pip install -U langchain-openai
!pip install langchain

In [ ]:
# prompt: 어떻게 api_key를 넣는가?
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

# Load the environment variables from the .env file
load_dotenv(find_dotenv())

# Get the OpenAI API key from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
api_key = 'YOUR API KEY'

In [ ]:
chat=ChatOpenAI(temperature=0,
                openai_api_key=api_key,
                )

Custom Tools

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

template = """Question: {question}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

llm_chain = LLMChain(
    prompt = prompt,
    llm = chat
)

print(llm_chain.invoke("What are Tesla's revenue in 2022?"))

In [ ]:
!pip install langchain-community # Install the langchain_community package

In [ ]:
!pip install duckduckgo-search

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
search.invoke("Tesla stock price?")

In [ ]:
from langchain.tools import Tool

duckduckgo_tool = Tool(
    name = 'DuckDuckGo Search',
    func = search.run,
    description = "Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

In [ ]:
from langchain.agents import AgentType
from langchain.agents import load_tools, initialize_agent

tools = [duckduckgo_tool]

agent = initialize_agent(
    tools,
    chat,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

In [ ]:
agent.run("What are Tesla's revenue in 2022?")

ChatGPT as a Financial Information Extractor

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

extraction_template = """Your task:\n\
Find the value of revenue in the given content.\n\
If you can't find the value, please output "None".\n\

Example 1:\n\
The amount of Apple's annual revenue in 2021 was $365.817B.
Result: 365.817

Given content: {text}
Result:
"""
extraction_prompt = ChatPromptTemplate.from_template(extraction_template)

extraction_chain = LLMChain(
    prompt = extraction_prompt,
    llm = chat
)

In [ ]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains = [agent, extraction_chain],
                                      verbose = True)

overall_chain.run("What are Tesla's revenue in 2022?")

In [ ]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains = [agent, extraction_chain],
                                      verbose = True)

overall_chain.run("테슬러의 2022년 매출은 얼마인가?")

# 연습문제

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

template = """Your task:\n\
Find the value of Scope 1 emissions in the given content.\n\
If you can't find the value, please output "None".\n\

Example 1:\n\
TotalEnergies' latest Scope 1 emissions were 32 million metric tons\
 of carbon dioxide in 2021.
Result: 32.0

Given content: {text}
Result:
"""
prompt = ChatPromptTemplate.from_template(template)

llm_chain = LLMChain(
    prompt = prompt,
    llm = chat
)

In [ ]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains = [agent, llm_chain],
    verbose = True
)

In [ ]:
from tqdm.notebook import tqdm

list_companies = ['AT&T',
 'Apple Inc.',
 'Bank of America',
 'Boeing',
 'CVS Health',
 'Chevron Corporation',
 'Cisco',
 'Citigroup',
 'Disney',
 'Dominion Energy',
 'ExxonMobil',
 'Ford Motor Company',
 'General Electric',
 'Home Depot (The)',
 'IBM',
 'Intel',
 'JPMorgan Chase',
 'Johnson & Johnson',
 "Kellogg's",
 'McKesson',
 'Merck & Co.',
 'Microsoft',
 'Oracle Corporation',
 'Pfizer',
 'Procter & Gamble',
 'United Parcel Service',
 'UnitedHealth Group',
 'Verizon',
 'Walmart',
 'Wells Fargo']

list_results = []

for i in tqdm(range(len(list_companies))):
  try:
    response = overall_chain.run(f"What is the amount of {list_companies[i]} Scope 1 emissions?")
  except:
    response = "None"
  list_results.append(response)

In [ ]:
list_results

Sustainability Report as a Knowledge Base

In [ ]:
!pip install unstructured
!pip install pypdf
!pip install pdf2image

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import OnlinePDFLoader

url = "https://www.tesla.com/ns_videos/2021-tesla-impact-report.pdf"
loader = OnlinePDFLoader(url)
data = loader.load()

In [ ]:
!pip install docarray
!pip install tiktoken

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
index.query("What is the Scope 1 Emissions?")